5，数据库问答(Querying Tabular Data): 从数据库/类数据库内容中抽取数据信息。

In [3]:
# 使用你自己的OpenAI API key
# openai_api_key='sk-PFwMvMFcU7VXaL5cYM9vT3B'
# zhipu_api_key='08132b620644081b192567639fc0a9b1.OAz1XI2d21zLZDk9'
#调用环境变量
import os
zhipu_api_key = os.environ.get('ZHIPU_API_KEY')

In [4]:
import time
import jwt
 
def generate_token(apikey: str, exp_seconds: int):
    try:
        id, secret = apikey.split(".")
    except Exception as e:
        raise Exception("invalid apikey", e)
 
    payload = {
        "api_key": id,
        "exp": int(round(time.time() * 1000)) + exp_seconds * 1000,
        "timestamp": int(round(time.time() * 1000)),
    }
 
    return jwt.encode(
        payload,
        secret,
        algorithm="HS256",
        headers={"alg": "HS256", "sign_type": "SIGN"},
    )

In [5]:
# from langchain_community.chat_models import ChatOpenAI
# from langchain.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI
llm =ChatOpenAI(
        model_name="glm-4",
        openai_api_base="https://open.bigmodel.cn/api/paas/v4",
        openai_api_key=generate_token(zhipu_api_key,10000),
        streaming=False,
        temperature=0.5
    )

## 五，数据库问答(Querying Tabular Data)

In [6]:
# 使用自然语言查询一个 SQLite 数据库，我们将使用旧金山树木数据集
# Don't run following code if you don't run sqlite and follow db
from langchain import OpenAI, SQLDatabase
# %pip install  --user langchain_experimental
from langchain_experimental.sql import SQLDatabaseChain
# llm = OpenAI(temperature=0, openai_api_key=openai_api_key)

In [7]:
sqlite_db_path = 'San_Francisco_Trees.db'
db = SQLDatabase.from_uri(f"sqlite:///{sqlite_db_path}")

In [8]:
db_chain = SQLDatabaseChain.from_llm(llm=llm, db=db, use_query_checker=True,verbose=True)

In [12]:
# db_chain.invoke("How many Species of trees are there in San Francisco?")
db_chain.invoke("多少树在San Francisco?")



> Entering new SQLDatabaseChain chain...
多少树在San Francisco?
SQLQuery:
 llm_inputs {'input': '多少树在San Francisco?\nSQLQuery:', 'top_k': '5', 'dialect': 'sqlite', 'table_info': '\nCREATE TABLE "SFTrees" (\n\t"TreeID" INTEGER, \n\t"qSpecies" TEXT, \n\t"qAddress" TEXT, \n\t"PlantType" TEXT\n)\n\n/*\n3 rows from SFTrees table:\nTreeID\tqSpecies\tqAddress\tPlantType\n168225\tArbutus \'Marina\' :: Hybrid Strawberry Tree\t2547 Vallejo St\tTree\n168228\tArbutus \'Marina\' :: Hybrid Strawberry Tree\t2547 Vallejo St\tTree\n189160\tAfrocarpus gracilior :: Fern Pine\t100 Kissling St\tTree\n*/', 'stop': ['\nSQLResult:']}

 sql_cmd SELECT COUNT("TreeID") FROM "SFTrees"
SQLResult:
query_checker_prompt:: input_variables=['dialect', 'query'] template='\n{query}\nDouble check the {dialect} query above for common mistakes, including:\n- Using NOT IN with NULL values\n- Using UNION when UNION ALL should have been used\n- Using BETWEEN for exclusive ranges\n- Data type mismatch in predicates\n- Properly qu

{'query': '多少树在San Francisco?', 'result': 'San Francisco有197,096棵树。'}

1. Find which table to use
2. Find which column to use
3. Construct the correct sql query
4. Execute that query
5. Get the result
6. Return a natural language reponse back

confirm LLM result via pandas